# DIT NLP lesson 2024

## Generating text with LSTMs

In [ ]:
# Run only the first time
import nltk
nltk.download('gutenberg')

In [ ]:
import numpy as np
import random
import sys

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

# In previous versions of Keras, the import was...
# from keras.optimizers import RMSprop

In [ ]:
from nltk.corpus import gutenberg
gutenberg.fileids()

Go to [Project Gutenberg](https://www.gutenberg.org) if you want more

In [ ]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()
chars = sorted(list(set(text)))

# dictionary from character to index
char_indices = dict((c, i) for i, c in enumerate(chars))

# distionary from index to character
indices_char = dict((i, c) for i, c in enumerate(chars))

print('corpus length: {} total chars: {}'.format(len(text), len(chars)))

print(text[:500])

**Objective.** Predicting the character after having seen 40 characters

**Trick.** Adding redundancy to the training collection

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
# Notice: no tokenisation; no sentence splitting; no linebreak elimination
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Total number of sequences:', len(sentences))

In [ ]:
# How does an instance look like?

idx = [0, 5, 2555, 10000, 12365]

for i in idx:
    print(i)
    print(sentences[i], " --> ", next_chars[i])
    print("="*50)

In [ ]:
# Producing the one-hot encoding
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
# Building the model

model = Sequential()
model.add(LSTM(128,
    input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# https://keras.io/api/optimizers/rmsprop/; learning rate default: 0.001
# learning_rate used to be lr in previous versions
optimizer = RMSprop(learning_rate=0.01)

# no more binary cross entropy; no dropout (we "kind of" want to overfit here)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()


In [ ]:
# Training
epochs = 6
batch_size = 128
model_structure = model.to_json()

with open("shakes_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
        batch_size=batch_size,
        epochs=epochs
    )
    model.save_weights("shakes_lstm_{}.weights.h5".format(i+1))

# It should take about 25 epochs

**Temperature**

temperature > 1 : more diverse outcome

temperature < 1 : more strict (try to "copy")

In [ ]:
def sample(preds, temperature=1.0):
    """Sampler to generate character sequences"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # produces a number of random outcomes,
    # given a probability distribution
    # n=1    number of experiments
    # preds  sequence of probabilities
    # size=1
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)
    # Getting a random starting text
    sentence = text[start_index: start_index + maxlen]
    generated = ''
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    for i in range(400):
        # one-hot representation
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        # Producing the prediction
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)

        # looking up the next character and adding it
        next_char = indices_char[next_index]
        generated += next_char

        #updating the seed
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()  # to display it right away
    print()

# lower values should look "more Shakesperean"

That's **70,200** parameters in the LSTM (against **17,550** for the RNN)

Back to the slides

## Homework

1. Try with shorter/longer contextual sequences
2. Build a model that try to mimic Dante

**End of the notebook**